In [358]:
import os
import plaid
import requests
import datetime
import json
import pandas as pd
%matplotlib inline

In [359]:
def pretty_print_response(response):
  print(json.dumps(response, indent=4, sort_keys=True))

In [360]:
PLAID_CLIENT_ID = os.getenv('PLAID_CLIENT_ID')
PLAID_SECRET = os.getenv('PLAID_SBX_SECRET_KEY')
PLAID_PUBLIC_KEY = os.getenv('PLAID_PUBLIC_KEY')
PLAID_ENV = os.getenv('PLAID_DEVELOPMENT_KEY', 'sandbox')
PLAID_PRODUCTS = os.getenv('PLAID_PRODUCTS', 'transactions')


In [361]:
client = plaid.Client(client_id=PLAID_CLIENT_ID, 
                      secret=PLAID_SECRET, 
                      public_key=PLAID_PUBLIC_KEY, 
                      environment='sandbox')


# Plaid Access Token

In this section, you will use the plaid-python api to generate the correct authentication tokens to access data in the free developer Sandbox. This mimics how you might connect to your own account or a customer account, but due to privacy issues, this homework will only require connecting to and analyzing the fake data from the developer sandbox that Plaid provides. 

Complete the following steps to generate an access token:
1. Create a client to connect to plaid
2. Use the client to generate a public token and request the following items: 
['transactions', 'income', 'assets']
3. Exchange the public token for an access token
4. Test the access token by requesting and printing the available test accounts

### 1. Create a client to connect to plaid

In [362]:
INSTITUTION_ID = "ins_109508"

In [363]:
api_key = PLAID_PUBLIC_KEY

api_request = 'https://www.plaid.com/api/'

### 2. Generate a public token

In [364]:
create_tkn_response = client.Sandbox.public_token.create(INSTITUTION_ID, 
                                                         ['transactions',
                                                          'income',
                                                          'assets'])


### 3. Exchange the public token for an access token


In [365]:
exchange_response = client.Item.public_token.exchange(create_tkn_response
                                                      ['public_token'])


### 4. Fetch Accounts

In [366]:
access_token = exchange_response["access_token"]

-----------

# Account Transactions with Plaid

In this section, you will use the Plaid Python SDK to connect to the Developer Sandbox account and grab a list of transactions. You will need to complete the following steps:


1. Use the access token to fetch the transactions for the last 90 days
2. Print the categories for each transaction type
3. Create a new DataFrame using the following fields from the JSON transaction data: `date, name, amount, category`. (For categories with more than one label, just use the first category label in the list)
4. Convert the data types to the appropriate types (i.e. datetimeindex for the date and float for the amount)

### 1. Fetch the Transactions for the last 90 days

In [367]:
start_date = f'{datetime.datetime.now() + datetime.timedelta(-90):%Y-%m-%d}'
end_date = f'{datetime.datetime.now():%Y-%m-%d}' 

transactions_response = client.Transactions.get(access_token, start_date, end_date)

print (json.dumps(transactions_response, indent = 4, sort_keys = True))


{
    "accounts": [
        {
            "account_id": "rbJR7Gjj8BIvavZXvn6wCeWg8r1KneClvqV38",
            "balances": {
                "available": 100,
                "current": 110,
                "iso_currency_code": "USD",
                "limit": null,
                "unofficial_currency_code": null
            },
            "mask": "0000",
            "name": "Plaid Checking",
            "official_name": "Plaid Gold Standard 0% Interest Checking",
            "subtype": "checking",
            "type": "depository"
        },
        {
            "account_id": "z3mMlGnnyjI3Z3yg3w5KH5ZnEQr4x5foGJ14P",
            "balances": {
                "available": 200,
                "current": 210,
                "iso_currency_code": "USD",
                "limit": null,
                "unofficial_currency_code": null
            },
            "mask": "1111",
            "name": "Plaid Saving",
            "official_name": "Plaid Silver Standard 0.1% Interest Saving",
       

### 2. Print the categories for each transaction


In [368]:
category_df = pd.DataFrame()
for category in transactions_response['transactions']:
    print(category['category'][:1])


['Travel']
['Food and Drink']
['Payment']
['Food and Drink']
['Shops']
['Payment']
['Travel']
['Transfer']
['Transfer']
['Recreation']
['Travel']
['Food and Drink']
['Food and Drink']
['Food and Drink']
['Transfer']
['Travel']
['Travel']


### 3. Create a new DataFrame using the following fields from the JSON transaction data: date, name, amount, category. 

(For categories with more than one label, just use the first category label in the list)

In [369]:
transaction_date = []
transaction_name = []
transaction_amount = []
transaction_category = []

for item in transactions_response['transactions']:
    transaction_date.append(item['date'])
    transaction_name.append(item['name'])
    transaction_amount.append(item['amount'])
    transaction_category.append(item['category'][:1])

In [370]:
date = pd.Series(transaction_date)
name = pd.Series(transaction_name)
category = pd.Series(transaction_category)
amount = pd.Series(transaction_amount)

In [371]:
transaction_data_df = pd.concat([date,name,amount,category], axis='columns', join='inner')
transaction_data_df.head()

,0,1,2,3
0,2020-01-29,Uber,6.33,[Travel]
1,2020-01-26,Tectra Inc,500.00,[Food and Drink]
2,2020-01-25,AUTOMATIC PAYMENT - THANK,2078.50,[Payment]
3,2020-01-25,KFC,500.00,[Food and Drink]
4,2020-01-25,Madison Bicycle Shop,500.00,[Shops]


In [372]:
transaction_data_df.columns = ['Date','Name','Amount','Category']
transaction_data_df.set_index(transaction_data_df['Date'], inplace = True)
transaction_data_df.drop(columns = ['Date'], inplace = True)
transaction_data_df.shape
transaction_data_df

,Name,Amount,Category
Date,,,
2020-01-29,Uber,6.33,[Travel]
2020-01-26,Tectra Inc,500.00,[Food and Drink]
2020-01-25,AUTOMATIC PAYMENT - THANK,2078.50,[Payment]
2020-01-25,KFC,500.00,[Food and Drink]
2020-01-25,Madison Bicycle Shop,500.00,[Shops]
2020-01-16,CREDIT CARD 3333 PAYMENT *//,25.00,[Payment]
2020-01-16,Uber,5.40,[Travel]
2020-01-15,ACH Electronic CreditGUSTO PAY 123456,5850.00,[Transfer]
2020-01-15,CD DEPOSIT .INITIAL.,1000.00,[Transfer]


### 4. Convert the data types to the appropriate types 

(i.e. datetimeindex for the date and float for the amount)

In [373]:
transaction_data_df.dtypes

Name         object
Amount      float64
Category     object
dtype: object

----------

# Income Analysis with Plaid

In this section, you will use the Plaid Sandbox to complete the following:
1. Determine the previous year's gross income and print the results
2. Determine the current monthly income and print the results
3. Determine the projected yearly income and print the results

In [375]:
income_response = client.Income.get(access_token)

#print (json.dumps(income_response, indent = 4, sort_keys = True))

In [376]:
print (f"Last year's gross income: {income_response['income']['last_year_income']}")
print (f"Current monthly income: {income_response['income']['income_streams'][0]['monthly_income']}")
print (f"Projected yearly income: {income_response['income']['projected_yearly_income']}")


Last year's gross income: 6000
Current monthly income: 500
Projected yearly income: 6085


-------

# Budget Analysis
In this section, you will use the transactions DataFrame to analyze the customer's budget

1. Calculate the total spending per category and print the results (Hint: groupby or count transactions per category)
2. Generate a bar chart with the number of transactions for each category 
3. Calculate the expenses per month
4. Plot the total expenses per month

### Calculate the expenses per category

In [379]:
transaction_data_df_modify = transaction_data_df.drop(columns = ['Name'], inplace = True)

transaction_data_df

KeyError: "['Name'] not found in axis"

In [378]:
transaction_data_df.groupby('Category', axis = 1)

ValueError: cannot reindex from a duplicate axis

### Calculate the expenses per month